In [ ]:
pip install tensorflow opencv-python spotipy numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
# Import required libraries
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.applications import MobileNetV2
import requests
from collections import defaultdict
import random
import time

def js_to_image(js_reply):
    """Convert JS image capture to OpenCV format"""
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def take_photo(filename='photo.jpg', quality=0.8):
    """Capture photo from webcam"""
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    return js_to_image(data)

class EmotionMusicRecommender:
    def __init__(self, lastfm_api_key):
        print("Initializing EmotionMusicRecommender...")

        # Initialize emotion model
        print("Loading emotion detection model...")
        self.emotion_model = self._build_emotion_model()

        # Initialize Last.fm
        print("Setting up Last.fm API...")
        self.lastfm_api_key = lastfm_api_key
        self.lastfm_base_url = "http://ws.audioscrobbler.com/2.0/"

        # Test Last.fm connection
        try:
            self._make_lastfm_request(method="chart.getTopArtists", limit=1)
            print("✅ Last.fm connection verified!")
        except Exception as e:
            print(f"❌ Last.fm connection error: {str(e)}")
            raise

        # Emotion mappings with Last.fm tags
        self.emotion_tags = {
            'happy': ['happy', 'upbeat', 'fun', 'party', 'energetic'],
            'sad': ['sad', 'melancholic', 'slow', 'emotional', 'ballad'],
            'angry': ['angry', 'aggressive', 'metal', 'intense', 'hardcore'],
            'neutral': ['chill', 'relaxing', 'ambient', 'smooth', 'easy'],
            'surprised': ['upbeat', 'electronic', 'dance', 'exciting', 'energetic'],
            'fearful': ['dark', 'atmospheric', 'intense', 'dramatic', 'epic'],
            'disgusted': ['intense', 'aggressive', 'dark', 'heavy', 'experimental']
        }

        self.recommendation_cache = defaultdict(list)
        print("Initialization complete! ✨")

    def _build_emotion_model(self):
        """Build emotion detection model"""
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        base_model.trainable = False

        model = Sequential([
            base_model,
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Dropout(0.25),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(7, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def _make_lastfm_request(self, method, **params):
        """Make a request to Last.fm API"""
        params.update({
            'method': method,
            'api_key': self.lastfm_api_key,
            'format': 'json'
        })

        response = requests.get(self.lastfm_base_url, params=params)
        response.raise_for_status()
        return response.json()

    def detect_emotion(self, frame):
        """Detect emotion from video frame"""
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None

        (x, y, w, h) = max(faces, key=lambda face: face[2] * face[3])
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = tf.keras.applications.mobilenet_v2.preprocess_input(
            np.expand_dims(face_roi, axis=0)
        )

        emotions = ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
        prediction = self.emotion_model.predict(face_roi, verbose=0)
        emotion = emotions[np.argmax(prediction)]
        confidence = np.max(prediction)

        return {'emotion': emotion, 'confidence': confidence}

    def get_music_recommendations(self, emotion, limit=5):
        """Get music recommendations based on emotion using Last.fm"""
        try:
            print(f"\n🎵 Finding {emotion} music...")

            # Get random tags for this emotion
            tags = random.sample(self.emotion_tags[emotion], 2)
            tag_string = ",".join(tags)
            print(f"Using tags: {tag_string}")

            # Get tag-based recommendations
            results = self._make_lastfm_request(
                method="tag.getTopTracks",
                tag=random.choice(tags),
                limit=limit
            )

            # Format recommendations
            recommendations = []
            for track in results['tracks']['track']:
                recommendations.append({
                    'name': track['name'],
                    'artist': track['artist']['name'],
                    'url': track['url']
                })

            return recommendations

        except Exception as e:
            print(f"\n⚠️ Error getting recommendations: {str(e)}")
            return []

    def run_emotion_detection(self):
        """Main loop for emotion detection and music recommendation"""
        print("\n🎵 Welcome to Emotion-Based Music Recommender! 🎵")
        print("Each photo will analyze your emotion and suggest music to match.")

        try:
            while True:
                print("\n📸 Capturing image... Click 'Capture' when ready!")
                frame = take_photo()

                print("\n🔍 Analyzing emotion...")
                emotion_result = self.detect_emotion(frame)

                if emotion_result:
                    emotion = emotion_result['emotion']
                    confidence = emotion_result['confidence']

                    print(f"\n😊 Detected emotion: {emotion.upper()} (confidence: {confidence:.2f})")

                    if confidence > 0.4:
                        recommendations = self.get_music_recommendations(emotion)

                        if recommendations:
                            print("\n🎧 Recommended songs based on your emotion:")
                            for i, rec in enumerate(recommendations, 1):
                                print(f"\n{i}. {rec['name']} by {rec['artist']}")
                                print(f"   🔗 Listen on Last.fm: {rec['url']}")
                        else:
                            print("\n❌ Could not find recommendations. Trying again...")
                            # Second attempt with different tags
                            time.sleep(1)  # Add delay to avoid rate limiting
                            recommendations = self.get_music_recommendations(emotion)
                            if recommendations:
                                print("\n🎧 Second attempt - Recommended songs:")
                                for i, rec in enumerate(recommendations, 1):
                                    print(f"\n{i}. {rec['name']} by {rec['artist']}")
                                    print(f"   🔗 Listen on Last.fm: {rec['url']}")
                            else:
                                print("\n❌ Still no recommendations found. Please try another photo.")
                    else:
                        print("\n⚠️ Low confidence in emotion detection. Please try another photo.")
                else:
                    print("\n❌ No face detected. Please ensure your face is clearly visible!")

                user_input = input("\n📸 Press Enter to capture another photo or 'q' to quit: ")
                if user_input.lower() == 'q':
                    break

        except Exception as e:
            print(f"\n❌ An error occurred: {str(e)}")
            print("Error details:", str(e))

        print("\n👋 Thank you for using Emotion-Based Music Recommender!")

# Your Last.fm API key
LASTFM_API_KEY = "a6eb8040a05bc20040d9d733a01590c4"  # Replace with your Last.fm API key

# Create and run the recommender
recommender = EmotionMusicRecommender(LASTFM_API_KEY)
recommender.run_emotion_detection()

Initializing EmotionMusicRecommender...
Loading emotion detection model...
Setting up Last.fm API...
✅ Last.fm connection verified!
Initialization complete! ✨

🎵 Welcome to Emotion-Based Music Recommender! 🎵
Each photo will analyze your emotion and suggest music to match.

📸 Capturing image... Click 'Capture' when ready!


<IPython.core.display.Javascript object>


🔍 Analyzing emotion...

😊 Detected emotion: HAPPY (confidence: 0.51)

🎵 Finding happy music...
Using tags: party,fun

🎧 Recommended songs based on your emotion:

1. Hey Ya! by OutKast
   🔗 Listen on Last.fm: https://www.last.fm/music/OutKast/_/Hey+Ya%21

2. Last Friday Night (T.G.I.F.) by Katy Perry
   🔗 Listen on Last.fm: https://www.last.fm/music/Katy+Perry/_/Last+Friday+Night+(T.G.I.F.)

3. A-Punk by Vampire Weekend
   🔗 Listen on Last.fm: https://www.last.fm/music/Vampire+Weekend/_/A-Punk

4. espresso by Sabrina Carpenter
   🔗 Listen on Last.fm: https://www.last.fm/music/Sabrina+Carpenter/_/espresso

5. We Are Young (feat. Janelle Monáe) by fun.
   🔗 Listen on Last.fm: https://www.last.fm/music/fun./_/We+Are+Young+(feat.+Janelle+Mon%C3%A1e)

📸 Press Enter to capture another photo or 'q' to quit: q

👋 Thank you for using Emotion-Based Music Recommender!


In [ ]:
!pip install ytmusicapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Import required libraries
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.applications import MobileNetV2
from ytmusicapi import YTMusic
from collections import defaultdict
import random
import time

def js_to_image(js_reply):
    """Convert JS image capture to OpenCV format"""
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def take_photo(filename='photo.jpg', quality=0.8):
    """Capture photo from webcam"""
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    return js_to_image(data)

class EmotionMusicRecommender:
    def __init__(self):
        print("Initializing EmotionMusicRecommender...")

        # Initialize emotion model
        print("Loading emotion detection model...")
        self.emotion_model = self._build_emotion_model()

        # Initialize YouTube Music
        print("Setting up YouTube Music API...")
        self.ytmusic = YTMusic()

        # Test YouTube Music connection
        try:
            self.ytmusic.search("test", filter="songs", limit=1)
            print("✅ YouTube Music connection verified!")
        except Exception as e:
            print(f"❌ YouTube Music connection error: {str(e)}")
            raise

        # Emotion mappings with Indian music keywords
        self.emotion_keywords = {
            'happy': [
                'happy bollywood songs', 'upbeat hindi songs',
                'punjabi bhangra songs', 'celebration hindi songs',
                'dance bollywood hits'
            ],
            'sad': [
                'sad hindi songs', 'emotional bollywood songs',
                'hindi sad love songs', 'emotional ghazals',
                'heartbreak bollywood songs','telugu sad love songs'
            ],
            'angry': [
                'powerful bollywood songs', 'aggressive hindi songs',
                'rock hindi songs', 'intense bollywood songs',
                'motivational hindi songs'
            ],
            'neutral': [
                'peaceful hindi songs', 'relaxing bollywood songs',
                'meditation indian music', 'soft hindi songs',
                'soothing bollywood music'
            ],
            'surprised': [
                'energetic bollywood songs', 'fusion indian songs',
                'modern hindi songs', 'trending bollywood hits',
                'new bollywood remixes'
            ],
            'fearful': [
                'dramatic bollywood songs', 'suspense hindi songs',
                'thriller bollywood music', 'intense indian songs',
                'dark mood bollywood'
            ],
            'disgusted': [
                'intense bollywood songs', 'powerful hindi songs',
                'strong bollywood tracks', 'dark mood hindi songs',
                'dramatic indian music'
            ]
        }

        self.recommendation_cache = defaultdict(list)
        print("Initialization complete! ✨")

    def _build_emotion_model(self):
        """Build emotion detection model"""
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        base_model.trainable = False

        model = Sequential([
            base_model,
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Dropout(0.25),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(7, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def detect_emotion(self, frame):
        """Detect emotion from video frame"""
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None

        (x, y, w, h) = max(faces, key=lambda face: face[2] * face[3])
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = tf.keras.applications.mobilenet_v2.preprocess_input(
            np.expand_dims(face_roi, axis=0)
        )

        emotions = ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
        prediction = self.emotion_model.predict(face_roi, verbose=0)
        emotion = emotions[np.argmax(prediction)]
        confidence = np.max(prediction)

        return {'emotion': emotion, 'confidence': confidence}

    def get_music_recommendations(self, emotion, limit=5):
        """Get Indian music recommendations based on emotion using YouTube Music"""
        try:
            print(f"\n🎵 Finding {emotion} Indian music...")

            # Get random keywords for this emotion
            search_query = random.choice(self.emotion_keywords[emotion])
            print(f"Using search query: {search_query}")

            # Search for songs
            results = self.ytmusic.search(search_query, filter="songs", limit=limit)

            # Format recommendations
            recommendations = []
            for song in results:
                if song.get('resultType') == 'song':
                    recommendations.append({
                        'name': song.get('title', ''),
                        'artist': ', '.join([artist['name'] for artist in song.get('artists', [])]),
                        'album': song.get('album', {}).get('name', ''),
                        'duration': song.get('duration', ''),
                        'url': f"https://music.youtube.com/watch?v={song.get('videoId')}"
                    })

            return recommendations[:limit]

        except Exception as e:
            print(f"\n⚠️ Error getting recommendations: {str(e)}")
            return []

    def run_emotion_detection(self):
        """Main loop for emotion detection and music recommendation"""
        print("\n🎵 Welcome to Indian Music Emotion Recommender! 🎵")
        print("Each photo will analyze your emotion and suggest Indian songs to match.")

        try:
            while True:
                print("\n📸 Capturing image... Click 'Capture' when ready!")
                frame = take_photo()

                print("\n🔍 Analyzing emotion...")
                emotion_result = self.detect_emotion(frame)

                if emotion_result:
                    emotion = emotion_result['emotion']
                    confidence = emotion_result['confidence']

                    print(f"\n😊 Detected emotion: {emotion.upper()} (confidence: {confidence:.2f})")

                    if confidence > 0.4:
                        recommendations = self.get_music_recommendations(emotion)

                        if recommendations:
                            print("\n🎧 Recommended Indian songs based on your emotion:")
                            for i, rec in enumerate(recommendations, 1):
                                print(f"\n{i}. {rec['name']}")
                                print(f"   👤 Artist: {rec['artist']}")
                                print(f"   💿 Album: {rec['album']}")
                                print(f"   ⏱️ Duration: {rec['duration']}")
                                print(f"   🔗 Listen: {rec['url']}")
                        else:
                            print("\n❌ Could not find recommendations. Trying again...")
                            # Second attempt with different keywords
                            time.sleep(1)  # Add delay to avoid rate limiting
                            recommendations = self.get_music_recommendations(emotion)
                            if recommendations:
                                print("\n🎧 Second attempt - Recommended songs:")
                                for i, rec in enumerate(recommendations, 1):
                                    print(f"\n{i}. {rec['name']}")
                                    print(f"   👤 Artist: {rec['artist']}")
                                    print(f"   💿 Album: {rec['album']}")
                                    print(f"   ⏱️ Duration: {rec['duration']}")
                                    print(f"   🔗 Listen: {rec['url']}")
                            else:
                                print("\n❌ Still no recommendations found. Please try another photo.")
                    else:
                        print("\n⚠️ Low confidence in emotion detection. Please try another photo.")
                else:
                    print("\n❌ No face detected. Please ensure your face is clearly visible!")

                user_input = input("\n📸 Press Enter to capture another photo or 'q' to quit: ")
                if user_input.lower() == 'q':
                    break

        except Exception as e:
            print(f"\n❌ An error occurred: {str(e)}")
            print("Error details:", str(e))

        print("\n👋 Thank you for using Indian Music Emotion Recommender!")

# First install the required package


# Create and run the recommender
recommender = EmotionMusicRecommender()
recommender.run_emotion_detection()

Initializing EmotionMusicRecommender...
Loading emotion detection model...
Setting up YouTube Music API...
✅ YouTube Music connection verified!
Initialization complete! ✨

🎵 Welcome to Indian Music Emotion Recommender! 🎵
Each photo will analyze your emotion and suggest Indian songs to match.

📸 Capturing image... Click 'Capture' when ready!


<IPython.core.display.Javascript object>


🔍 Analyzing emotion...

😊 Detected emotion: SURPRISED (confidence: 0.48)

🎵 Finding surprised Indian music...
Using search query: fusion indian songs

🎧 Recommended Indian songs based on your emotion:

1. Indian Flute Music: Instrumental Meditation Music, Yoga Spa Music and Relaxation
   👤 Artist: Nature Sounds
   💿 Album: Indian Flute Music: Instrumental Meditation Music, Yoga Spa Music and Relaxation
   ⏱️ Duration: 1:57:59
   🔗 Listen: https://music.youtube.com/watch?v=5F_lxaJkXUM

2. Divine Indian Flute Music
   👤 Artist: Supernatural Brainwave Power
   💿 Album: Manifestation Sutra
   ⏱️ Duration: 1:00:26
   🔗 Listen: https://music.youtube.com/watch?v=91iGTV2HNbY

3. Tantra Flute (Indian Flute Meditation Music)
   👤 Artist: Meditative Mind
   💿 Album: Tantra Flute (Indian Flute Meditation Music)
   ⏱️ Duration: 1:01:01
   🔗 Listen: https://music.youtube.com/watch?v=FUg-f_2S-ro

4. Bollywood Dj Non Stop Remix(Remix By Dj Jitesh,Psynth)
   👤 Artist: Bappi Lahiri, K.K., Anushka Manch

In [ ]:
var=